In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

# GroupBy

In [3]:
"""By “group by” we are referring to a process involving one or more of the following steps:
1) Splitting the data into groups based on some criteria.
2) Applying a function to each group independently.
3) Combining the results into a data structure."""

'By “group by” we are referring to a process involving one or more of the following steps:\n1) Splitting the data into groups based on some criteria.\n2) Applying a function to each group independently.\n3) Combining the results into a data structure.'

In [4]:
# A list or array of values that is the same length as the axis being grouped
# A value indicating a column name in a DataFrame
# A dict or Series giving a correspondence bet the values on the axis being grouped and the group names
# A function to be invoked on the axis index or the individual labels in the index

In [5]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a','b'], 'key2' : ['one', 'two', 'one', 'two', 'one','one'],
                   'data1' : np.random.randn(6),'data2' : np.random.randn(6)})

In [6]:
df

,key1,key2,data1,data2
0,a,one,0.482343,1.852250
1,a,two,-1.479404,-0.120297
2,b,one,-0.900853,-2.271522
3,b,two,-1.280723,-1.598456
4,a,one,1.877732,-0.018107
5,b,one,0.118492,-0.888943


In [7]:
# to compute the mean of the data1 column using the labels from key1.

In [8]:
grouped = df['data1'].groupby(df['key1'])

In [27]:
grouped

In [28]:
grouped.mean()

key1
a    0.107179
b    0.076151
Name: data1, dtype: float64

In [29]:
"""data (a Series) has been aggregated according to the group key, producing
a new Series that is now indexed by the unique values in the key1 column. The
result index has the name'key1' because the DataFrame column df['key1'] """ 

"data (a Series) has been aggregated according to the group key, producing\na new Series that is now indexed by the unique values in the key1 column. The\nresult index has the name'key1' because the DataFrame column df['key1'] "

In [17]:
df.groupby('key1')['data1']

In [18]:
df.groupby('key1')[['data2']]

In [19]:
df['data1'].groupby(df['key1'])

In [ ]:
df[['data2']].groupby(df['key1'])

In [ ]:
# column names as the group keys

In [8]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.646708,-0.470045
b,-0.653789,-0.382820


In [ ]:
# with multiple keys

In [9]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [11]:
means

key1  key2
a     one    -0.923332
      two    -0.093459
b     one    -0.156934
      two    -1.647499
Name: data1, dtype: float64

In [10]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.130684
     two  -1.148769
b    one  -0.621780
     two   0.095100

In [ ]:
"""Here we grouped the data using two keys, and the resulting Series
now has a hierarchical index consisting of the unique pairs of keys"""

In [ ]:
# multiple column names as the group keys

In [12]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.923332 -0.130684
     two  -0.093459 -1.148769
b    one  -0.156934 -0.621780
     two  -1.647499  0.095100

In [ ]:
# groupby with series as array with same length

In [13]:
city = np.array(['Hyd', 'Kol', 'Hyd', 'Pune', 'Pune','Kol'])

In [14]:
years = np.array([2016, 2016, 2017, 2016, 2017,2017])

In [15]:
df['data1'].groupby([city, years]).mean()

Hyd   2016   -1.503760
      2017   -0.490329
Kol   2016   -0.093459
      2017    0.176461
Pune  2016   -1.647499
      2017   -0.342904
Name: data1, dtype: float64

In [ ]:
# Size
#useful GroupBy method , which returns a Series containing group sizes
# any missing values in a group key will be excluded from the result

In [16]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     2
      two     1
dtype: int64

# Iterating Over Groups

In [ ]:
# The GroupBy object supports iteration, generating a sequence of 2-tuples
# containing the group name along with the chunk of data

In [17]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.503760  0.140029
1    a  two -0.093459 -1.148769
4    a  one -0.342904 -0.401396
b
  key1 key2     data1     data2
2    b  one -0.490329 -1.431467
3    b  two -1.647499  0.095100
5    b  one  0.176461  0.187908


In [ ]:
# with multiple keys

In [ ]:
# In the case of multiple keys, the first element in the tuple will
# be a tuple of key values

In [18]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2)) 
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.503760  0.140029
4    a  one -0.342904 -0.401396
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.093459 -1.148769
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.490329 -1.431467
5    b  one  0.176461  0.187908
('b', 'two')
  key1 key2     data1   data2
3    b  two -1.647499  0.0951


In [ ]:
# groupby computing on dict of the data pieces as a one-liner

In [19]:
pieces = dict(list(df.groupby('key1')))

In [20]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.490329,-1.431467
3,b,two,-1.647499,0.095100
5,b,one,0.176461,0.187908


In [ ]:
# groupby on axis1

In [21]:
grouped = df.groupby(df.dtypes, axis=1)

In [22]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.503760  0.140029
1 -0.093459 -1.148769
2 -0.490329 -1.431467
3 -1.647499  0.095100
4 -0.342904 -0.401396
5  0.176461  0.187908
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one
5    b  one


# Grouping with Dicts and Series

In [23]:
students = pd.DataFrame(np.random.randn(5, 5),columns=['a', 'b', 'c', 'd', 'e'],
          index=['Ram', 'Shyam', 'Mohan', 'Rohan', 'Sohan'])

In [24]:
students.iloc[2:3, [1, 2]] = np.nan

In [25]:
students

,a,b,c,d,e
Ram,0.698443,0.528551,1.427511,-0.377404,-0.280450
Shyam,-0.120138,-0.635510,-0.121573,-0.528837,-1.549739
Mohan,0.706939,NaN,NaN,-0.458724,-0.727294
Rohan,-0.280350,0.031698,0.206441,-1.546446,1.328752
Sohan,0.036567,0.762068,0.959423,0.971935,0.584202


In [27]:
diction = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [28]:
df_dict_grp_by = students.groupby(diction, axis=1)

In [29]:
df_dict_grp_by.sum()

,blue,red
Ram,1.050108,0.946544
Shyam,-0.650410,-2.305387
Mohan,-0.458724,-0.020355
Rohan,-1.340005,1.080099
Sohan,1.931358,1.382837


In [ ]:
# with series

In [30]:
w_series = pd.Series(diction)

In [31]:
w_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [32]:
students.groupby(w_series, axis=1).count()

,blue,red
Ram,2,3
Shyam,2,3
Mohan,1,2
Rohan,2,3
Sohan,2,3


# Grouping with Functions

In [ ]:
# Any function passed as a group key
# will be called once per index value, with the return values being
# used as the group names

In [ ]:
"""Suppose you wanted to group by the length of the names; while
you could compute an array of string lengths, it’s simpler to just pass
the len function """

In [34]:
students.groupby(len).sum()

,a,b,c,d,e
3,0.698443,0.528551,1.427511,-0.377404,-0.280450
5,0.343018,0.158256,1.044291,-1.562073,-0.364079


In [ ]:
"""Mixing functions with arrays, dicts, or Series is not a problem as
everything gets converted to arrays internally"""

In [36]:
_list = ['one', 'one', 'one', 'two', 'two']

In [37]:
students.groupby([len, _list]).min()

a         b         c         d         e
3 one  0.698443  0.528551  1.427511 -0.377404 -0.280450
5 one -0.120138 -0.635510 -0.121573 -0.528837 -1.549739
  two -0.280350  0.031698  0.206441 -1.546446  0.584202

# Grouping by Index Levels

In [38]:
col = pd.MultiIndex.from_arrays([['UP', 'UP', 'UP', 'HP', 'HP'],[1, 3, 5, 1, 3]],
                                names=['city', 'centre'])

In [39]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=col)

In [40]:
hier_df

city          UP                            HP          
centre         1         3         5         1         3
0      -0.456838 -0.187620 -0.543111  1.286439 -0.121009
1       0.084785 -0.532783  1.293451  0.147579 -0.520807
2       0.841682  0.862748 -1.911075 -0.811728 -0.007017
3      -1.124314  0.191227  1.144697  0.984360  0.026153

In [ ]:
#To group by level, pass the level number or name using the level keyword

In [41]:
hier_df.groupby(level='city', axis=1).count()

city,HP,UP
0,2,3
1,2,3
2,2,3
3,2,3


# Data Aggregation

In [ ]:
#Aggregations refer to any data transformation that produces scalar values from arrays

In [ ]:
"""Aggregation: compute a summary statistic (or statistics) for each group. Some examples:
Compute group sums or means.
Compute group sizes / counts."""

In [42]:
df

,key1,key2,data1,data2
0,a,one,-1.503760,0.140029
1,a,two,-0.093459,-1.148769
2,b,one,-0.490329,-1.431467
3,b,two,-1.647499,0.095100
4,a,one,-0.342904,-0.401396
5,b,one,0.176461,0.187908


In [43]:
grouped = df.groupby('key1')

In [44]:
grouped['data1'].quantile(0.9)

key1
a   -0.143348
b    0.043103
Name: data1, dtype: float64

In [1]:
# User Defined function

In [45]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [46]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.410301,1.288798
b,1.823960,1.619374


In [ ]:
# methods like describe also work, even though they are not aggregations

In [47]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.646708  0.752635 -1.503760 -0.923332 -0.342904 -0.218181   
b      3.0 -0.653789  0.922901 -1.647499 -1.068914 -0.490329 -0.156934   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.093459   3.0 -0.470045  0.647136 -1.148769 -0.775082 -0.401396   
b     0.176461   3.0 -0.382820  0.909340 -1.431467 -0.668183  0.095100   

                          
           75%       max  
key1                      
a    -0.130684  0.140029  
b     0.141504  0.187908

Inside GroupBy, when you invoke a method like describe, it is actually just a
shortcut of below function

In [48]:
f = lambda x: x.describe()
grouped.apply(f)

data1     data2
key1                          
a    count  3.000000  3.000000
     mean  -0.646708 -0.470045
     std    0.752635  0.647136
     min   -1.503760 -1.148769
     25%   -0.923332 -0.775082
     50%   -0.342904 -0.401396
     75%   -0.218181 -0.130684
     max   -0.093459  0.140029
b    count  3.000000  3.000000
     mean  -0.653789 -0.382820
     std    0.922901  0.909340
     min   -1.647499 -1.431467
     25%   -1.068914 -0.668183
     50%   -0.490329  0.095100
     75%   -0.156934  0.141504
     max    0.176461  0.187908

# Column-Wise and Multiple Function Application

In [49]:
tips = sns.load_dataset('tips')

In [50]:
tips.head(2)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3


In [ ]:
# Add tip percentage of total bill

In [51]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [52]:
tips[:5]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [53]:
grouped = tips.groupby(['day', 'smoker'])

In [54]:
grouped_pct = grouped['tip_pct']

In [55]:
grouped_pct.agg('mean')

day   smoker
Thur  Yes       0.163863
      No        0.160298
Fri   Yes       0.174783
      No        0.151650
Sat   Yes       0.147906
      No        0.158048
Sun   Yes       0.187250
      No        0.160113
Name: tip_pct, dtype: float64

If you pass a list of functions or function names instead, 
you get back a DataFrame with column names taken from the functions

In [56]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Thur Yes     0.163863  0.039389      0.151240
     No      0.160298  0.038774      0.193350
Fri  Yes     0.174783  0.051293      0.159925
     No      0.151650  0.028123      0.067349
Sat  Yes     0.147906  0.061375      0.290095
     No      0.158048  0.039767      0.235193
Sun  Yes     0.187250  0.154134      0.644685
     No      0.160113  0.042347      0.193226

 if you pass a list of (name,function) tuples, 
 the first element of each tuple will be used as the DataFrame column names

In [59]:
grouped_pct.agg([('ele1', 'mean'), ('ele2', np.std)])

ele1      ele2
day  smoker                    
Thur Yes     0.163863  0.039389
     No      0.160298  0.038774
Fri  Yes     0.174783  0.051293
     No      0.151650  0.028123
Sat  Yes     0.147906  0.061375
     No      0.158048  0.039767
Sun  Yes     0.187250  0.154134
     No      0.160113  0.042347

With a DataFrame you can specify a list of functions to apply to all of the columns 
or different functions per column

In [60]:
func = ['count', 'mean', 'max']

In [61]:
result = grouped['tip_pct', 'total_bill'].agg(func)

In [62]:
result                    # hierarchical index in o/p

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Thur Yes         17  0.163863  0.241255         17  19.190588  43.11
     No          45  0.160298  0.266312         45  17.113111  41.19
Fri  Yes         15  0.174783  0.263480         15  16.813333  40.17
     No           4  0.151650  0.187735          4  18.420000  22.75
Sat  Yes         42  0.147906  0.325733         42  21.276667  50.81
     No          45  0.158048  0.291990         45  19.661778  48.33
Sun  Yes         19  0.187250  0.710345         19  24.120000  45.35
     No          57  0.160113  0.252672         57  20.506667  48.17

In [63]:
result['tip_pct']

count      mean       max
day  smoker                           
Thur Yes        17  0.163863  0.241255
     No         45  0.160298  0.266312
Fri  Yes        15  0.174783  0.263480
     No          4  0.151650  0.187735
Sat  Yes        42  0.147906  0.325733
     No         45  0.158048  0.291990
Sun  Yes        19  0.187250  0.710345
     No         57  0.160113  0.252672

In [ ]:
# list of tuples with custom names

In [64]:
fun_tup = [('Doremon', 'mean'), ('Picachoo', np.var)]

In [65]:
grouped['tip_pct', 'total_bill'].agg(fun_tup)

tip_pct           total_bill            
              Doremon  Picachoo    Doremon    Picachoo
day  smoker                                           
Thur Yes     0.163863  0.001551  19.190588   69.808518
     No      0.160298  0.001503  17.113111   59.625081
Fri  Yes     0.174783  0.002631  16.813333   82.562438
     No      0.151650  0.000791  18.420000   25.596333
Sat  Yes     0.147906  0.003767  21.276667  101.387535
     No      0.158048  0.001581  19.661778   79.908965
Sun  Yes     0.187250  0.023757  24.120000  109.046044
     No      0.160113  0.001793  20.506667   66.099980

In [ ]:
# Apply potentially different functions to one or more of the columns

we need to pass a dict to agg that contains a mapping of column names to
any of the function specified

In [66]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Thur Yes      5.00    40
     No       6.70   112
Fri  Yes      4.73    31
     No       3.50     9
Sat  Yes     10.00   104
     No       9.00   115
Sun  Yes      6.50    49
     No       6.00   167

A DataFrame will have hierarchical columns only if multiple
functions are applied to at least one column

In [67]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Thur Yes     0.090014  0.241255  0.163863  0.039389   40
     No      0.072961  0.266312  0.160298  0.038774  112
Fri  Yes     0.103555  0.263480  0.174783  0.051293   31
     No      0.120385  0.187735  0.151650  0.028123    9
Sat  Yes     0.035638  0.325733  0.147906  0.061375  104
     No      0.056797  0.291990  0.158048  0.039767  115
Sun  Yes     0.065660  0.710345  0.187250  0.154134   49
     No      0.059447  0.252672  0.160113  0.042347  167

# Returning Aggregated Data Without Row Indexes

the aggregated data comes back with
an index, potentially hierarchical, composed from the unique group key
combinations

In [ ]:
#disable this behavior in most cases by passing as_index=False to groupby

In [68]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Thur,Yes,19.190588,3.030000,2.352941,0.163863
1,Thur,No,17.113111,2.673778,2.488889,0.160298
2,Fri,Yes,16.813333,2.714000,2.066667,0.174783
3,Fri,No,18.420000,2.812500,2.250000,0.151650
4,Sat,Yes,21.276667,2.875476,2.476190,0.147906
5,Sat,No,19.661778,3.102889,2.555556,0.158048
6,Sun,Yes,24.120000,3.516842,2.578947,0.187250
7,Sun,No,20.506667,3.167895,2.929825,0.160113


# Apply: General split-apply-combine

In [ ]:
# we want top 5 value of tip_pict in group

In [69]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [70]:
top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [ ]:
# group by smoker, and call apply with the above function

The top function will be called on each row group from the
DataFrame, and then the results are glued together using pandas.concat,
labeling the pieces with thegroup names. T
he result therefore has a hierarchical index whose inner
level contains index values from the original DataFrame.

In [71]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990

If you pass a function to apply that takes other arguments or keywords, you can
pass these after the
function

In [72]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
Yes    Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   
       Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
No     Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
       Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   

                  tip_pct  
smoker day                 
Yes    Thur 197  0.115982  
       Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
No     Thur 142  0.121389  
       Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799

# Suppressing the Group Keys

resulting object has a hierarchical
index
formed from the group keys along with the indexes of each piece of the
original object. You can disable this by passing group_keys=False to groupby

In [73]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990


# Quantile and Bucket Analysis

pandas has some tools,
in particular cut and qcut, for slicing data up into buckets with bins of your
choosing or
by sample quantiles. Combining these functions with groupby makes it
convenient to perform
bucket or quantile analysis on a dataset.

In [74]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

In [75]:
quartiles = pd.cut(frame.data1, 4)

In [76]:
quartiles[:10]

0     (1.893, 3.509]
1    (-1.339, 0.277]
2    (-1.339, 0.277]
3     (0.277, 1.893]
4     (0.277, 1.893]
5     (0.277, 1.893]
6     (0.277, 1.893]
7    (-1.339, 0.277]
8    (-1.339, 0.277]
9    (-1.339, 0.277]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.962, -1.339] < (-1.339, 0.277] < (0.277, 1.893] < (1.893, 3.509]]

In [ ]:
# The Categorical object
# returned by cut can be
# passed directly to groupby. So we
# could compute a set of statistics for the data2 column like

In [77]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

In [78]:
grouped = frame.data2.groupby(quartiles)

In [79]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.962, -1.339]",99.0,2.357877,-0.029652,-2.607451
"(-1.339, 0.277]",526.0,3.382438,0.026157,-2.447328
"(0.277, 1.893]",355.0,3.142896,0.043675,-2.926056
"(1.893, 3.509]",20.0,1.940176,-0.000198,-2.127457


In [ ]:
# Return quantile numbers, pass labels=False

In [80]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [81]:
grouped = frame.data2.groupby(grouping)

In [82]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.357877,-0.020524,-2.607451
1,100.0,2.798307,-0.077190,-2.263955
2,100.0,2.617783,0.083262,-1.980221
3,100.0,2.386408,0.026096,-2.369624
4,100.0,3.109044,0.075726,-2.128477
5,100.0,3.382438,0.042998,-2.285191
6,100.0,2.418510,-0.161945,-2.447328
7,100.0,2.449590,0.141116,-2.926056
8,100.0,3.142896,0.097440,-2.798870


# Pivot table

A pivot table is a data summarization tool
frequently found in spreadsheet programs and other data
analysis software. It aggregates a table of data by one or more keys,
arranging the data in a rectangle with some of the group keys along the
rows and some along the columns

In [83]:
tips.pivot_table(index=['day', 'smoker'])  # similar to groupby

size       tip   tip_pct  total_bill
day  smoker                                          
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333

In [84]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker            Yes        No       Yes        No
time   day                                         
Dinner Sun   2.578947  2.929825  0.187250  0.160113
       Sat   2.476190  2.555556  0.147906  0.158048
       Thur       NaN  2.000000       NaN  0.159744
       Fri   2.222222  2.000000  0.165347  0.139622
Lunch  Thur  2.352941  2.500000  0.163863  0.160311
       Fri   1.833333  3.000000  0.188937  0.187735

In [ ]:
# by passing  margins=True. 
# This will add All row and column labels, 
# with corresponding values being the group statistics for all the data within a single tier
# resulting all values AS MEAN

In [85]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)   

size                       tip_pct                    
smoker            Yes        No       All       Yes        No       All
time   day                                                             
Dinner Sun   2.578947  2.929825  2.842105  0.187250  0.160113  0.166897
       Sat   2.476190  2.555556  2.517241  0.147906  0.158048  0.153152
       Thur       NaN  2.000000  2.000000       NaN  0.159744  0.159744
       Fri   2.222222  2.000000  2.166667  0.165347  0.139622  0.158916
Lunch  Thur  2.352941  2.500000  2.459016  0.163863  0.160311  0.161301
       Fri   1.833333  3.000000  2.000000  0.188937  0.187735  0.188765
All          2.408602  2.668874  2.569672  0.163196  0.159328  0.160803

In [86]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

day            Thur   Fri   Sat   Sun    All
time   smoker                               
Dinner No       1.0   3.0  45.0  57.0  106.0
       Yes      NaN   9.0  42.0  19.0   70.0
Lunch  No      44.0   1.0   NaN   NaN   45.0
       Yes     17.0   6.0   NaN   NaN   23.0
All            62.0  19.0  87.0  76.0  244.0

In [ ]:
# Replacing Null Value

In [87]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                     Thur       Fri       Sat       Sun
time   size smoker                                        
Dinner 1    No      0.000000  0.000000  0.137931  0.000000
            Yes     0.000000  0.000000  0.325733  0.000000
       2    No      0.159744  0.139622  0.162705  0.168859
            Yes     0.000000  0.171297  0.148668  0.207893
       3    No      0.000000  0.000000  0.154661  0.152663
            Yes     0.000000  0.000000  0.144995  0.152660
       4    No      0.000000  0.000000  0.150096  0.148143
            Yes     0.000000  0.117750  0.124515  0.193370
       5    No      0.000000  0.000000  0.000000  0.206928
            Yes     0.000000  0.000000  0.106572  0.065660
       6    No      0.000000  0.000000  0.000000  0.103799
Lunch  1    No      0.181728  0.000000  0.000000  0.000000
            Yes     0.000000  0.223776  0.000000  0.000000
       2    No      0.166005  0.000000  0.000000  0.000000
            Yes     0.158843  0.181969  0.000000  0.000000
       3    No      0.084246  0.187735  0.000000  0.000000
            Yes     0.204952  0.000000  0.000000  0.000000
       4    No      0.138919  0.000000  0.000000  0.000000
            Yes     0.155410  0.000000  0.000000  0.000000
       5    No      0.121389  0.000000  0.000000  0.000000
       6    No      0.173706  0.000000  0.000000  0.000000

# Cross-Tabulations: Crosstab

A cross-tabulation (or crosstab for short) is a special
case of a pivot table that computes group frequencies.

In [88]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Sun    57   19   76
       Sat    45   42   87
       Thur    1    0    1
       Fri     3    9   12
Lunch  Thur   44   17   61
       Fri     1    6    7
All          151   93  244

In [ ]:
data = pd.DataFrame({'City':['Hyd','Blore','Hyd','Blore','Blore','Blore',
                                    'Hyd','Hyd','Blore','Hyd'] 
                     'Handedness' : ['Right-handed','Left-handed','Right-handed','Right-handed',
                                     'Left-handed','Right-handed','Right-handed',
                                     'Left-handed','Right-handed','Right-handed']})

In [ ]:
pd.crosstab(data.City, data.Handedness, margins=True)